<a href="https://colab.research.google.com/github/qdrptd/RNE/blob/main/YTCrawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/

In [2]:
from selenium import webdriver
import time
from openpyxl import Workbook
import pandas as pd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [6]:
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
import requests
!pip install xlsxwriter
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
driver.quit()

In [ ]:
#해당 url로 이동
pd_metadata = {}
pd_comments = pd.DataFrame({})
for t in range(3, 4):
  type_n = open('type'+str(t)+".txt").readlines()
  search_data = {}
  for search in type_n:
    driver = webdriver.Chrome('chromedriver', options=options)
    url = "https://www.youtube.com/results?search_query=" + search #search 단어를 검색했을 때 나오는 페이지
    driver.get(url)
    driver.execute_script("window.scrollTo(0,1000)")
    time.sleep(3)

    # 페이지 끝까지 스크롤(복사)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    print(last_height)
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.5)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

        if new_height > 100000: #동영상은 꽤 많이 나와서, 페이지 길이에 한계를 설정
          break

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    video_list = soup.select("a#video-title")

    id, titles = [], []
    for video in video_list: 
      id.append(video['href']) #동영상 ID(링크) 
      titles.append(video['title']) #동영상 제목
    views, channels, subs = [], [], []
    channel_id_list = soup.select('div#channel-info > a')
    views_list = soup.select("div#metadata > div > span:nth-of-type(1)")
    time_list = soup.select("div#metadata > div > span:nth-of-type(2)")
    # for channel_id in channel_id_list:
    #   try:
    #     channels.append(channel_id['href'])
    #     channel_url = "https://www.youtube.com" + channel_id['href']
    #     channel_html = requests.get(channel_url).text
    #     channel_soup = BeautifulSoup(channel_html, 'html.parser')
    #     subs.append(channel_soup.find('#subscriber-count'))
    #   except:
    #     pass
      
    for v in views_list:
      views.append(v.text)
    # search_data[search] = {"아이디" : id, "제목" : titles, "조회수": views, "채널": channels, "구독자": subs}
    id_final = []
    comment_final = []
    video_comments_data = {}
    for video_id in id:
      url = "https://www.youtube.com" + video_id
      try:
        driver.get(url)
      except:
        break
      driver.implicitly_wait(3)

      time.sleep(1.5)

      driver.execute_script("window.scrollTo(0,1000)")
      time.sleep(3)

      # 페이지 끝까지 스크롤
      last_height = driver.execute_script("return document.documentElement.scrollHeight")

      while True:
          driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
          time.sleep(1.5)

          new_height = driver.execute_script("return document.documentElement.scrollHeight")
          if new_height == last_height:
              break
          last_height = new_height
          if new_height > 100000:
            break

      time.sleep(1.5)

      # 팝업 닫기
      try:
          driver.find_element_by_css_selector("#dismiss-button > a").click()
      except:
          pass

      # 대댓글 모두 열기
      buttons = driver.find_elements_by_css_selector("#more-replies > a")

      time.sleep(1.5)

      for button in buttons:
          button.send_keys(Keys.ENTER)
          time.sleep(1.5)
          try:
            button.click()
          except:
            pass

      time.sleep(1.5)

      # 정보 추출하기
      html_source = driver.page_source
      soup = BeautifulSoup(html_source, 'html.parser')

      id_list = soup.select("div#header-author > h3 > #author-text > span")
      comment_list = soup.select("yt-formatted-string#content-text")
      for i in range(len(comment_list)):
        temp_id = id_list[i].text
        temp_id = temp_id.replace('\n', '')
        temp_id = temp_id.replace('\t', '')
        temp_id = temp_id.replace('    ', '')
        id_final.append(temp_id)

        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        comment_final.append(temp_comment)
      pd_comments = pd.concat([pd_comments, pd.DataFrame({'타입': t, '검색어': search,'영상': video_id,'작성자': id_final, '내용': comment_final})])
      print("종료")
  print()
      
  # pd_metadata['type' + str(i)] = search_data
print(pd_comments)
pd.DataFrame(pd_comments).to_excel("comments.xlsx")
# pd_metadata = pd.DataFrame(pd_metadata)
# pd_metadata.to_excel('metadata.xlsx')
driver.quit()

5577


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:93: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead


종료
